# Multiphase Systems

Kawin supports the usage of multiple phases. Nucleation and growth rate are handled for each precipitate phase independently. Coupling comes from the mass balance where all precipitates contribute to the overall mass changes in the system.

In the Al-Mg-Si system, several phases can form including: $ \beta' $, $ \beta" $, B', U1 and U2. To model precipitation of these phases, they must be defined in the .tdb file, the Thermodynamics module and the PrecipitateModel module.

When defining the thermodynamics module, the first phase in the list of phases will be the parent phase.

In [2]:
from kawin.thermo import MulticomponentThermodynamics

phases = ['FCC_A1', 'MGSI_B_P', 'MG5SI6_B_DP', 'B_PRIME_L', 'U1_PHASE', 'U2_PHASE']
therm = MulticomponentThermodynamics('AlMgSi.tdb', ['AL', 'MG', 'SI'], phases, drivingForceMethod='approximate')

In defining the precipitate model, all precipitate phases must be included. Since we already have our list of phases, we can use that and remove the parent phase.

In [3]:
from kawin.precipitation import PrecipitateModel, VolumeParameter

model = PrecipitateModel(phases=phases[1:], elements=['MG', 'SI'])

## Model inputs

Setting up parameters for the parent phase and overall system is the same as for single phase systems. Here, it is just the composition (Al-0.72Mg-0.57Si in mol. %), molar volume ($1e$-$5\text{ }m^3/mol$).

The temperature will be divided into two stages: a 16 hour temper at $175\text{ }^oC$, followed by a 1 hour ramp up to $250 ^oC$. To do this, there needs to be three time designations: $175\text{ }^oC$ at 0 hours, $175\text{ }^oC$ at 16 hours and $250\text{ }^oC$ at 17 hours. The temperature can be plotted to show the profile over time. Here, a parameter called timeUnits is passed to convert the time from seconds to either minutes or hours.

In [4]:
%matplotlib inline
import matplotlib.pyplot as plt

model.setInitialComposition([0.0072, 0.0057])
model.setVolumeAlpha(1e-5, VolumeParameter.MOLAR_VOLUME, 4)

lowTemp = 175+273.15
highTemp = 250+273.15
model.setTemperature(([0, 16, 17], [lowTemp, lowTemp, highTemp]))

Setting parameters for each precipitate phase is similar to single phase systems except that the phase has to be defined when inputting parameters.

In [5]:
gamma = {
    'MGSI_B_P': 0.18,
    'MG5SI6_B_DP': 0.084,
    'B_PRIME_L': 0.18,
    'U1_PHASE': 0.18,
    'U2_PHASE': 0.18
        }

for i in range(len(phases)-1):
    model.setInterfacialEnergy(gamma[phases[i+1]], phase=phases[i+1])
    model.setVolumeBeta(1e-5, VolumeParameter.MOLAR_VOLUME, 4, phase=phases[i+1])
    model.setThermodynamics(therm, phase=phases[i+1])

## Solving the model

As with single precipitate phase systems, running the model is exactly the same.

kawin currently implements two iterative methods for solving a model: Explicit euler and 4th order Runga Kutta. The Runga Kutta method is used by default, but we can input a different iterative method when solving. Here, we'll use explicit euler to have the model solve a bit faster.
- Another note: the solverType parameter in the solve function can take in either a SolverType enumerator or an Iterator from kawin.solver.Iterator which allows for custom iteration schemes that are not yet implemented in kawin to be used.

In [6]:
from kawin.solver import SolverType

model.solve(25*3600, solverType=SolverType.EXPLICITEULER, verbose=True, vIt=5000)

Nucleation density not set.
Setting nucleation density assuming grain size of 100 um and dislocation density of 5e+12 #/m2
N	Time (s)	Sim Time (s)	Temperature (K)	MG	SI	
0	0.0e+00		0.0		448		0.7200	0.5700	

	Phase	Prec Density (#/m3)	Volume Frac	Avg Radius (m)	Driving Force (J/mol)
	MGSI_B_P	0.000e+00		0.0000		0.0000e+00	1.2935e+04
	MG5SI6_B_DP	0.000e+00		0.0000		0.0000e+00	6.4812e+03
	B_PRIME_L	0.000e+00		0.0000		0.0000e+00	8.0247e+03
	U1_PHASE	0.000e+00		0.0000		0.0000e+00	7.5291e+03
	U2_PHASE	0.000e+00		0.0000		0.0000e+00	7.1709e+03



C:\Users\ury3\OneDrive - LLNL\Documents\Projects\pycalphad\pycalphad\pycalphad\core\utils.py:54: RuntimeWarning: invalid value encountered in divide
  pts[:, cur_idx:end_idx] /= pts[:, cur_idx:end_idx].sum(axis=1)[:, None]
C:\Users\ury3\OneDrive - LLNL\Documents\Projects\U-C Modeling\kawin-development\kawin\kawin\precipitation\KWNBase.py:1216: RuntimeWarning: divide by zero encountered in scalar divide
  return np.exp(-tau / (t - self.time[startIndex]))


N	Time (s)	Sim Time (s)	Temperature (K)	MG	SI	
5000	6.1e+04		104.6		523		0.0616	0.2060	

	Phase	Prec Density (#/m3)	Volume Frac	Avg Radius (m)	Driving Force (J/mol)
	MGSI_B_P	2.059e+22		1.0251		4.8766e-09	7.2355e+02
	MG5SI6_B_DP	6.383e+06		0.0000		3.9642e-10	-4.6030e+03
	B_PRIME_L	3.851e+15		0.0000		1.0417e-09	-2.0841e+03
	U1_PHASE	0.000e+00		0.0000		0.0000e+00	1.2718e+03
	U2_PHASE	0.000e+00		0.0000		0.0000e+00	-5.1915e+02

N	Time (s)	Sim Time (s)	Temperature (K)	MG	SI	
10000	6.1e+04		198.8		523		0.0620	0.2062	

	Phase	Prec Density (#/m3)	Volume Frac	Avg Radius (m)	Driving Force (J/mol)
	MGSI_B_P	2.059e+22		1.0246		4.8708e-09	7.3894e+02
	MG5SI6_B_DP	0.000e+00		0.0000		0.0000e+00	-4.5911e+03
	B_PRIME_L	2.011e+04		0.0000		1.1245e-09	-2.0724e+03
	U1_PHASE	0.000e+00		0.0000		0.0000e+00	1.2777e+03
	U2_PHASE	0.000e+00		0.0000		0.0000e+00	-5.1063e+02

N	Time (s)	Sim Time (s)	Temperature (K)	MG	SI	
13565	9.0e+04		256.3		523		0.0566	0.2032	

	Phase	Prec Density (#/m3)	Volume Frac	Avg Radius (m)

## Plotting

Plotting is also the same as with single phase systems. The major difference is each phase will be plotted for the radius, volume fraction, precipitate density, nucleation rate and particle size distribution. In addition, the total amount of some variables, such as the precipitate density and volume fraction, can be plotted.

In [7]:
fig, axes = plt.subplots(2, 2, figsize=(10, 8))

model.plot(axes[0,0], 'Total Precipitate Density', timeUnits='h', label='Total', color='k', linestyle=':', zorder=6)
model.plot(axes[0,0], 'Precipitate Density', timeUnits='h')
axes[0,0].set_ylim([1e5, 1e25])
axes[0,0].set_xscale('linear')
axes[0,0].set_yscale('log')

model.plot(axes[0,1], 'Total Volume Fraction', timeUnits='h', label='Total', color='k', linestyle=':', zorder=6)
model.plot(axes[0,1], 'Volume Fraction', timeUnits='h')
axes[0,1].set_xscale('linear')

model.plot(axes[1,0], 'Average Radius', timeUnits='h')
axes[1,0].set_xscale('linear')

model.plot(axes[1,1], 'Composition', timeUnits='h')
axes[1,1].set_xscale('linear')

fig.tight_layout()

## References

1. E. Povoden-Karadeniz et al, "Calphad modeling of metastable phases in the Al-Mg-Si system" *Calphad* 43 (2013) p. 94
2. Q. Du et al, "Modeling over-ageing in Al-Mg-Si alloys by a multi-phase Calphad-coupled Kampmann-Wagner Numerical model" *Acta Materialia* 122 (2017) p. 178